In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [4]:
EPOCHS = 40
BATCH_SIZE = 64

In [11]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,),(0.3081,))
                         ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train=False,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,),(0.3081,))
                         ])),
    batch_size=BATCH_SIZE, shuffle=True)

In [23]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #10개의 특징맵 생성
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        
#         # 함수 말고 모듈 사용
#         self.drop = nn.Dropout2d()
        
#         # ( 입력크기(앞 계층의 출력 크기), 출력크기 )
#         self.fc1 = nn.Linear(320, 50) 
#         self.fc2 = nn.Linear(50, 10)
        
#         # x가 conv 계층 2개를 거친 특징 맵이 되는 과정 
#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2)) #(  , 커널 크기)
#         x = F.relu(F.max_pool2d(self.conv2(x), 2)) #2차원

#         #특징맵 이후의 출력하는 일반 인공신경망은 1차원의 입력을 받음
#         x = x.view(-1, 320) #(남는 차원 모두, x가 가진 원소 개수)

#         x = F.relu(self.fc1(x))
#         x = self.drop(x)
#         x = self.fc2(x)
#         return F.log_softmax(x, dim=1)

# 88.46%

In [42]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #10개의 특징맵 생성
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50) 
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) #(  , 커널 크기)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) #2차원
        x = x.view(-1, 320) #(남는 차원 모두, x가 가진 원소 개수)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [43]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)

In [44]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))

In [45]:
def evaluate(model, test_loader):
    model.eval() # 모델을 평가모드로
    test_loss = 0
    correct = 0
    with torch.no_grad(): # 평가 과정에서는 기울기를 계산하지 않아도 됨
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target, 
                                        reduction='sum').item()
                                    # sum : 미니배치 평균 대신 합을 받아옴
                                    # item : 1개의 값을 가진 텐서 값 받아옴
                    
            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [31]:
# for epoch in range(1, EPOCHS + 1):
#     train(model, train_loader, optimizer, epoch)
#     test_loss, test_accuracy = evaluate(model, test_loader)
    
#     print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
#             epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss:0.607165
Train Epoch: 1 [12800/60000 (21%)]	Loss:0.603055
Train Epoch: 1 [25600/60000 (43%)]	Loss:0.538478
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.461455
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.433100
[1] Test Loss: 0.3926, Accuracy: 85.52%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.440599
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.309783
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.468855
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.513326
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.474307
[2] Test Loss: 0.3656, Accuracy: 87.03%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.329250
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.412361
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.502122
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.265741
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.250490
[3] Test Loss: 0.3499, Accuracy: 87.30%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.403356
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.502929
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.403708
Train Epoch: 4 [38

[29] Test Loss: 0.3409, Accuracy: 87.88%
Train Epoch: 30 [0/60000 (0%)]	Loss:0.371368
Train Epoch: 30 [12800/60000 (21%)]	Loss:0.347238
Train Epoch: 30 [25600/60000 (43%)]	Loss:0.392866
Train Epoch: 30 [38400/60000 (64%)]	Loss:0.286064
Train Epoch: 30 [51200/60000 (85%)]	Loss:0.238078
[30] Test Loss: 0.3485, Accuracy: 88.42%
Train Epoch: 31 [0/60000 (0%)]	Loss:0.215928
Train Epoch: 31 [12800/60000 (21%)]	Loss:0.373818
Train Epoch: 31 [25600/60000 (43%)]	Loss:0.363160
Train Epoch: 31 [38400/60000 (64%)]	Loss:0.218137
Train Epoch: 31 [51200/60000 (85%)]	Loss:0.403097
[31] Test Loss: 0.3608, Accuracy: 88.34%
Train Epoch: 32 [0/60000 (0%)]	Loss:0.238217
Train Epoch: 32 [12800/60000 (21%)]	Loss:0.152182
Train Epoch: 32 [25600/60000 (43%)]	Loss:0.240706
Train Epoch: 32 [38400/60000 (64%)]	Loss:0.433219
Train Epoch: 32 [51200/60000 (85%)]	Loss:0.469790
[32] Test Loss: 0.3282, Accuracy: 88.17%
Train Epoch: 33 [0/60000 (0%)]	Loss:0.272106
Train Epoch: 33 [12800/60000 (21%)]	Loss:0.207415
Train 

In [46]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
            epoch, test_loss, test_accuracy))
    
# 2차 => 성능 떨어졌음

Train Epoch: 1 [0/60000 (0%)]	Loss:2.309143
Train Epoch: 1 [12800/60000 (21%)]	Loss:0.748090
Train Epoch: 1 [25600/60000 (43%)]	Loss:0.811618
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.583721
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.797115
[1] Test Loss: 0.4917, Accuracy: 80.65%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.627998
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.706860
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.680342
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.652387
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.687596
[2] Test Loss: 0.4161, Accuracy: 84.19%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.372743
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.616275
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.506039
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.574165
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.470309
[3] Test Loss: 0.4224, Accuracy: 83.77%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.522601
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.576433
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.764095
Train Epoch: 4 [38

[29] Test Loss: 0.3648, Accuracy: 86.25%
Train Epoch: 30 [0/60000 (0%)]	Loss:0.730208
Train Epoch: 30 [12800/60000 (21%)]	Loss:0.521627
Train Epoch: 30 [25600/60000 (43%)]	Loss:0.509742
Train Epoch: 30 [38400/60000 (64%)]	Loss:0.442640
Train Epoch: 30 [51200/60000 (85%)]	Loss:0.623262
[30] Test Loss: 0.3547, Accuracy: 86.85%
Train Epoch: 31 [0/60000 (0%)]	Loss:0.300100
Train Epoch: 31 [12800/60000 (21%)]	Loss:0.630804
Train Epoch: 31 [25600/60000 (43%)]	Loss:0.370319
Train Epoch: 31 [38400/60000 (64%)]	Loss:0.363892
Train Epoch: 31 [51200/60000 (85%)]	Loss:0.316892
[31] Test Loss: 0.3617, Accuracy: 86.49%
Train Epoch: 32 [0/60000 (0%)]	Loss:0.647037
Train Epoch: 32 [12800/60000 (21%)]	Loss:0.506413
Train Epoch: 32 [25600/60000 (43%)]	Loss:0.418803
Train Epoch: 32 [38400/60000 (64%)]	Loss:0.488066
Train Epoch: 32 [51200/60000 (85%)]	Loss:0.554419
[32] Test Loss: 0.3689, Accuracy: 86.03%
Train Epoch: 33 [0/60000 (0%)]	Loss:0.709598
Train Epoch: 33 [12800/60000 (21%)]	Loss:0.608890
Train 

In [ ]:
# output.max() : 가장 큰 값, 인덱스(자리)
# eq() : 0/1 => sum()
# veiw_as : target 텐서를 view_as() 함수 안에 들어가는 
#          인수(여기서 pred)의 모양대로 다시 정렬
# item() 예제?